In [28]:
import pandas as pd
import numpy as np
import math

In [2]:
data=pd.read_csv('GKX_20201231.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

# 1. Select data from 1960 to 2019

In [ ]:
data = data[data.DATE >= 19600101][data.DATE <=20191231].reset_index(drop = True)

In [ ]:
data.head()

In [ ]:
data.tail()

In [7]:
data['DATE']=data['DATE']//100 
data = data.rename(columns={'DATE': 'yyyymm'})#Let the DATE become the format "yyyymm"

# 2. Construct full dataset

In [9]:
dm=pd.read_excel('macro.xlsx')

In [ ]:
dm.head()

In [ ]:
dm = dm.loc[(dm['yyyymm']>=196001)&(dm['yyyymm']<202001)]
dm

## 2.1 Calculate some macropredictors that are not given directly
       Dividen to ptice ratio(dp):log(D12)-log(Index)
       Earnings-price ratio(ep):log(E12)-log(Index)
       Term spread(tms):lty-tbl
       Default spread(dfy):BAA-AAA

In [13]:
dm['dp']=dm['D12'].apply(math.log)-dm['Index'].apply(math.log)
dm['ep']=dm['E12'].apply(math.log)-dm['Index'].apply(math.log)
dm['tms']=dm['lty']-dm['tbl']
dm['dfy']=dm['BAA']-dm['AAA']
#Rename b/m by bm and Keep only 8 needed macropredictors
ned=['yyyymm', 'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy','svar']
dm=dm.rename(columns={'b/m':'bm'})[ned]
dm.to_csv('macro.csv',index=False)

## 2.2 Join the two datasets

In [14]:
macro = pd.read_csv('macro.csv')
macro


,yyyymm,dp,ep,bm,ntis,tbl,tms,dfy,svar
0,196001,-3.394191,-2.797533,0.499502,0.022116,0.0435,0.0006,0.0073,0.000919
1,196002,-3.383903,-2.806662,0.493557,0.024037,0.0396,0.0033,0.0078,0.001150
2,196003,-3.350808,-2.792666,0.549798,0.025593,0.0331,0.0080,0.0076,0.000969
3,196004,-3.331425,-2.787838,0.563404,0.025577,0.0323,0.0103,0.0075,0.000645
4,196005,-3.356176,-2.827389,0.541966,0.024414,0.0329,0.0088,0.0082,0.000424
...,...,...,...,...,...,...,...,...,...
715,201908,-3.941330,-3.086025,0.248539,-0.010142,0.0195,-0.0032,0.0089,0.004318
716,201909,-3.951689,-3.108987,0.243797,-0.010838,0.0189,-0.0019,0.0088,0.000605
717,201910,-3.965984,-3.112869,0.242630,-0.013157,0.0165,0.0006,0.0091,0.001510
718,201911,-3.993568,-3.130267,0.233936,-0.007796,0.0154,0.0027,0.0088,0.000306


In [15]:
data1 = pd.merge(data, macro, how='left', on='yyyymm', suffixes=('', '_macro'))

In [16]:
data1['excess_ret']=data1['RET']-data1['tbl'] #Calculate return in excess of risk-free rate

## 2.3 Fill the missing value with cross-sectional median or 0

In [18]:
nonpd = ['yyyymm','RET','SHROUT','mve0','prc','permno','excess_ret'] #Nonpredictors
macropd = ['dp','ep_macro','bm_macro','ntis','tbl','tms','dfy','svar'] #Macroeconomic predictors
sic2 = ['sic2'] #Industrial dummies
lst = nonpd+macropd+sic2
stockpd = [p for p in data1.columns if p not in lst] #Stock-level predictors
len(stockpd)

94

In [19]:
period_list = np.unique(data1['yyyymm'])
len(period_list)

720

In [ ]:
for pd in stockpd:
    c_row = 0
    for period in period_list:
        n_period = data1[data1['yyyymm']==period].shape[0]
        #Compute the cross-sectional median of this stock predictor
        median = np.nanmedian(data1.loc[c_row:(c_row+n_period-1),pd])
        #Fill missing values with median
        data1.loc[c_row:(c_row+n_period-1),pd] = data1.loc[c_row:(c_row+n_period-1),pd].replace(np.nan,median)
        c_row = c_row + n_period

In [23]:
newdt = data1.fillna(0)

In [ ]:
newdt = newdt.drop(columns=['prc','SHROUT','mve0','RET']) #Drop the predictors that are not used


In [25]:
newdt.to_csv('merge_data.csv', index = False)